In [ ]:
import os
os.environ["NUMBA_NUM_THREADS"] = "20"

import netket as nk
import netket.experimental as nkx

from vmc_torch.fermion_utils import generate_random_fpeps
import quimb.tensor as qtn
import symmray as sr
import autoray as ar


# Define the lattice shape
Lx = 4
Ly = 4
spinless = False
# graph = nk.graph.Square(L)
graph = nk.graph.Grid([Lx,Ly], pbc=False)
N = graph.n_nodes

# Define the fermion filling and the Hilbert space
N_f = int(Lx*Ly-2)
n_fermions_per_spin = (N_f//2, N_f//2)
hi = nkx.hilbert.SpinOrbitalFermions(N, s=1/2, n_fermions_per_spin=n_fermions_per_spin)

# SU in quimb
D = 4
seed = 0
symmetry = 'U1'
spinless = False
peps = generate_random_fpeps(Lx, Ly, D=D, seed=seed, symmetry=symmetry, Nf=N_f, spinless=spinless)[0]
edges = qtn.edges_2d_square(Lx, Ly, cyclic=False)
try:
    parse_edges_to_site_info = sr.utils.parse_edges_to_site_info
except AttributeError:
    parse_edges_to_site_info = sr.parse_edges_to_site_info
site_info = parse_edges_to_site_info(
    edges,
    D,
    phys_dim=4,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

t = 1.0
U = 8.0
if N_f == int(Lx*Ly-2) or N_f == int(Lx*Ly-8):
    mu = 0.0 if symmetry == 'U1' else (U*N_f/(2*N)-2.42)#(U*N_f/(2*N)-2.3)
elif N_f == int(Lx*Ly):
    mu = 0.0 if symmetry == 'U1' else (U/2)
elif N_f == int(Lx*Ly-4):
    mu = 0.0 if symmetry == 'U1' else (U/2)-U*0.3
else:
    mu = 0.0

print(mu)

terms = {
    (sitea, siteb): sr.fermi_hubbard_local_array(
        t=t, U=U, mu=mu,
        symmetry=symmetry,
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    )
    for (sitea, siteb) in peps.gen_bond_coos()
}
N_terms = {
    site: sr.fermi_number_operator_spinful_local_array(
        symmetry=symmetry
    )
    for site in peps.gen_site_coos()
}
occ_fn = lambda su: print(f'N per site:{su.get_state().compute_local_expectation(N_terms, normalized=True, max_bond=64,)/N}') if su.n%50==0 else None

ham = qtn.LocalHam2D(Lx, Ly, terms)

su = qtn.SimpleUpdateGen(peps, ham, compute_energy_per_site=True,D=D, compute_energy_opts={"max_distance":1}, gate_opts={'cutoff':1e-12}, callback=occ_fn)

# cluster energies may not be accuracte yet
su.evolve(50, tau=0.3)
# su.evolve(50, tau=0.1)
# su.evolve(50, tau=0.03)
# # su.evolve(50, tau=0.01)
# # su.evolve(50, tau=0.003)

peps = su.get_state()
peps.equalize_norms_(value=1)

# save the state
params, skeleton = qtn.pack(peps)


0.0


n=50, tau=0.3, max|dS|=0.15: 100%|##########| 50/50 [00:06<00:00,  8.02it/s] 

N per site:0.8749999093994323


n=50, tau=0.3, max|dS|=0.15, energy~-0.533988: 100%|##########| 50/50 [00:09<00:00,  5.44it/s]


In [ ]:

u1array = peps.arrays[0]
u1indices = u1array.indices
u1charge = u1array.charge
u1blocks = u1array.blocks
u1oddpos = u1array.oddpos
# u1array.to_dense()
# sr.Z2FermionicArray.from_dense(u1array.to_dense())
u1duals = u1array.duals
# ar.shape(u1array.to_dense())
u1array.to_dense()[3][2][0], u1blocks
def u1ind_to_z2indmap(u1indices):
    index_maps = []
    for blockind in u1indices:
        index_map = {}
        indicator = 0 #max value=blocind.size_total-1
        for c, dim in blockind.chargemap.items():
            for i in range(indicator, indicator+dim):
                index_map[i]=int(c%2)
            indicator+=dim
        index_maps.append(index_map)
    return index_maps
# index_map1={0:1,1:0,2:0,3:1}
# index_map2={0:1,1:0,2:0,3:1}
# index_map3={0:0,1:1,2:1,3:0}

# index_maps = [index_map1, index_map2, index_map3]
index_maps = u1ind_to_z2indmap(u1indices)

z2array=sr.Z2FermionicArray.from_dense(u1array.to_dense(), index_maps=index_maps, duals=u1duals, charge=u1charge, oddpos=u1oddpos)
def u1arr_to_z2arr(u1array):
    """
    Convert a FermionicArray with U1 symmetry to a FermionicArray with Z2 symmetry
    """
    u1indices = u1array.indices
    u1charge = u1array.charge
    u1oddpos = u1array.oddpos
    u1duals = u1array.duals
    index_maps = u1ind_to_z2indmap(u1indices)
    z2array=sr.Z2FermionicArray.from_dense(u1array.to_dense(), index_maps=index_maps, duals=u1duals, charge=u1charge%2, oddpos=u1oddpos)
    return z2array

def u1peps_to_z2peps(peps):
    """
    Convert a PEPS with U1 symmetry to a PEPS with Z2 symmetry
    """
    pepsu1 = peps.copy()
    for ts in pepsu1.tensors:
        ts.modify(data=u1arr_to_z2arr(ts.data))
    return pepsu1.copy()

z2array = u1arr_to_z2arr(u1array)
peps = su.get_state()
for ts in peps.tensors:
    ts.modify(data=u1arr_to_z2arr(ts.data))
pepsu1 = su.get_state()
pepsz2 = peps.copy()




In [145]:
from vmc_torch.hamiltonian import spinful_Fermi_Hubbard_square_lattice
import numpy as np
H = spinful_Fermi_Hubbard_square_lattice(Lx, Ly, t, U, N_f, pbc=False, n_fermions_per_spin=n_fermions_per_spin)
random_config = H.hilbert.random_state(np.random.randint(2**32-1))
# random_config = torch.tensor(random_config, dtype=dtype)
ampu1 = pepsu1.get_amp(random_config)
ampz2 = pepsz2.get_amp(random_config)
ampu1.contract(), ampz2.contract(), pepsu1.symmetry, pepsz2.symmetry

(1.6823871013676544e-07, np.float64(1.682387101367663e-07), <U1>, <Z2>)

In [149]:
N_terms = {
    site: sr.fermi_number_operator_spinful_local_array(
        symmetry='Z2'
    )
    for site in peps.gen_site_coos()
}
pepsz2.compute_local_expectation(N_terms, normalized=True, max_bond=64,)/N

np.float64(0.8749999093994322)

In [150]:
N_terms = {
    site: sr.fermi_number_operator_spinful_local_array(
        symmetry='U1'
    )
    for site in peps.gen_site_coos()
}
pepsu1.compute_local_expectation(N_terms, normalized=True, max_bond=64,)/N

np.float64(0.8749999093994323)